# Research Track 2 final assignment by G.A. Kurshakov

## How to run the code

Follow the steps:

- Build the ROS package using `catkin_make`
- Start the simulation running `roslaunch rt2_assignment1 sim.launch`
- Push the *Run all* button in this Jupyter Notebook to run the control node.
- Enjoy the interface and the plots!
> You might want to check if all the publishers and subscribers were initialized correctly running `rosnode info /robot_control_node` in your Linux terminal.

In this initial part we import all the packages we might need (a lot of them!) and initialize the global variables.

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
import math
import time
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from geometry_msgs.msg import Twist
from rt2_assignment1.srv import Command
from rt2_assignment1.msg import PositionActionResult
from tf import transformations
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox

rospy.init_node('robot_control_node')

goals = 0
cancelled = 0 
reached = 0 
active = False
start_time = time.time()
stop_time = 0
data = list()
odom_linear =list()
cmdvel_linear=list()
odom_angular =list()
cmdvel_angular=list()
time_arr = list(range(0, 10))
#time2 = list(range(0, 10))
clock = time.time()

## Input

### Buttons to start and stop robot random behavior

We initialize a user interface client to send our commands.

In [2]:
ui_client = rospy.ServiceProxy('/user_interface', Command)

Here we define three beautiful buttons for starting random behavior, stopping it and stopping it immediately.

In [3]:
start = Button(description='START',
layout=Layout(width='10%', align="center", grid_area='start'),
style=ButtonStyle(button_color='lightgreen'))

stop = Button(description='STOP',
layout=Layout(width='10%', align="center", grid_area='stop'),
style=ButtonStyle(button_color='moccasin'))

stop_now = Button(description='STOP NOW',
layout=Layout(width='10%', align="center", grid_area='stop_now'),
style=ButtonStyle(button_color='salmon'))

HBox([start, stop, stop_now])

And finally we define their behavior upon being clicked. Each button sends its own request to the *user_interface* server, corresponding to the requests in the original assignment solution.

In [4]:
def on_start_clicked(b):
    global active
    global start_time
    ui_client("start")
    active = True
    start_time = time.time()
    
def on_stop_clicked(b):
    ui_client("stop")
    
def on_stopnow_clicked(b):
    global active
    global cancelled
    ui_client("stop_now")
    active = False
    cancelled += 1
    
start.on_click(on_start_clicked)
stop.on_click(on_stop_clicked)
stop_now.on_click(on_stopnow_clicked)

### Sliders to change velocity

Sliders are directly linked to **ROS Parameter Server**. Every time a slider value is changed, the corresponding parameter is being set to the new value.

In [5]:
lin_vel = widgets.FloatSlider(min=0.0, max=2.0, value=1.0, description="Linear")
display(lin_vel)

ang_vel = widgets.FloatSlider(min=0.0, max=2.0, value=1.0, description="Angular")
display(ang_vel)

def on_valuelin_change(change):
    rospy.set_param('lin_vel', change['new'])
    
def on_valueang_change(change):
    rospy.set_param('ang_vel', change['new'])

lin_vel.observe(on_valuelin_change, names='value')
ang_vel.observe(on_valueang_change, names='value')

FloatSlider(value=1.0, description='Linear', max=2.0)

FloatSlider(value=1.0, description='Angular', max=2.0)

### Manual control buttons

The buttons are arranged in an intuitive fashion and augmented with the arrows of corresponding direction.

In [6]:
forward = Button(description='\u2191',
layout=Layout(width='auto', align="center", grid_area='forward'),
style=ButtonStyle(button_color='lightgreen'))

backward = Button(description='\u2193',
layout=Layout(width='auto', align="center", grid_area='backward'),
style=ButtonStyle(button_color='lightgreen'))

left = Button(description='\u21b6',
layout=Layout(width='auto', align="center", grid_area='left'),
style=ButtonStyle(button_color='lightgreen'))

right = Button(description='\u21b7',
layout=Layout(width='auto', align="center", grid_area='right'),
style=ButtonStyle(button_color='lightgreen'))

cancel = Button(description='O',
layout=Layout(width='auto', align="center", grid_area='cancel'),
style=ButtonStyle(button_color='lightgreen'))

GridBox(children=[forward, backward, left, right, cancel],
layout=Layout(
width='10%',
grid_template_rows='33% 33% 33%',
grid_template_columns='33% 33% 33%',
grid_template_areas='''
" . forward . "
"left cancel right "
" . backward . "
''')
)

GridBox(children=(Button(description='↑', layout=Layout(grid_area='forward', width='auto'), style=ButtonStyle(…

A publisher publishes a corresponding velocity command on */cmd_vel* topic each time a button is clicked. Hence, if you change the velocity using the sliders, it will not be changed until you click one of the control buttons again. When any of this button is clicked a "stop_now" message is being sent to the user interface server in case the robot is currently performing random behavior. This will stop the robot and after that you might need to click the button again in case stopping command will be published later than the corresponding velocity.

In [7]:
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

def on_forward_clicked(b):
    global active
    if active == True:
        global cancelled
        ui_client("stop_now")
        active = False
        cancelled += 1
    msg = Twist()
    msg.linear.x = rospy.get_param('lin_vel')
    pub.publish(msg)
    
def on_backward_clicked(b):
    global active
    if active == True:
        global cancelled
        ui_client("stop_now")
        active = False
        cancelled += 1
    msg = Twist()
    msg.linear.x = -rospy.get_param('lin_vel')
    pub.publish(msg)
    
def on_left_clicked(b):
    global active
    if active == True:
        global cancelled
        ui_client("stop_now")
        active = False
        cancelled += 1
    msg = Twist()
    msg.angular.z = -rospy.get_param('ang_vel')
    pub.publish(msg)
    
def on_right_clicked(b):
    global active
    if active == True:
        global cancelled
        ui_client("stop_now")
        active = False
        cancelled += 1
    msg = Twist()
    msg.angular.z = rospy.get_param('ang_vel')
    pub.publish(msg)
    
def on_cancel_clicked(b):
    global active
    if active == True:
        global cancelled
        ui_client("stop_now")
        active = False
        cancelled += 1
    msg = Twist()
    pub.publish(msg)
    
forward.on_click(on_forward_clicked)
backward.on_click(on_backward_clicked)
left.on_click(on_left_clicked)
right.on_click(on_right_clicked)
cancel.on_click(on_cancel_clicked)

## Output

### A position plot

The node is subscribed to */odom* topic. It takes position and appends it to a data vector corresponding to the trajectory. The orientation is received from the same topic, that is how we get the yaw angle to be depicted in the plot. Also from */odom* we get the linear and angular velocity data to be displayed further in the line plots.

In [8]:
fig1, ax1 = plt.subplots()
fig1.suptitle('Robot position and orientation plot', fontsize=16)

ax1.set_xlim(( -5, 5))
ax1.set_ylim((-5, 5))
ax1.set_aspect("equal")

line, = ax1.plot([], [], 'ro')
an1 = ax1.annotate("", xy=(0, 1), xytext=(0, 0), arrowprops={"facecolor": "black"})
x_data=[]
y_data=[]
yaw = 0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def init():
    line.set_data([], [])
    return (line, )


def odom_callback(msg):
    global yaw
    global clock
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw = euler[2]
    if len(time_arr) > 10:
        time_arr.pop(0)
    if len(odom_linear) > 10:
        odom_linear.pop(0)
    if len(odom_angular) > 10:
        odom_angular.pop(0)
    il = np.float32(msg.twist.twist.linear.x)
    ia = np.float32(msg.twist.twist.angular.z)
    it = np.float32(time.time() - clock)
    odom_linear.append(il)
    odom_angular.append(ia)
    time_arr.append(it)


def animate(i):
    global yaw
    line.set_data(x_data, y_data)
    an1.xy = (x_data[-1] + math.cos(yaw), y_data[-1] + math.sin(yaw))
    an1.set_position((x_data[-1], y_data[-1]))
    return (line, an1)



jr.subscribe('/odom', Odometry, odom_callback)

In [10]:
anim = animation.FuncAnimation(fig1, animate, init_func=init,
                               frames=100, interval=20, blit=True)

### A reached and cancelled targets bar plot

Each time a target is reached, the node receives the action result (both if the target was really reached or just cancelled). Cancelled targets are enumerated in the cancel buttons handlers. Therefore, we get the number of cancelled goals directly and can get the number of reached goals subtracting cancelled goals from the total number.
> This plot is not animated! You have to rerun the cell manually if you want to update it!

In [11]:
def reached_goal_clb(msg):
    global goals
    global reached
    global start_time
    global stop_time
    global data 
    stop_time = time.time()
    data.append(stop_time - start_time)
    start_time = time.time()
    goals += 1
    reached = goals - cancelled
  

jr.subscribe('/go_to_point/result',PositionActionResult, reached_goal_clb)

In [21]:
fig2, ax2 = plt.subplots()
fig2.suptitle('Number of targets reached and cancelled', fontsize=16)
y = (reached,cancelled)
index = np.arange(2)

fig2, ax2 = plt.bar(index,y,width=0.5,color='r')
plt.ylabel('Values')
plt.xticks(index, ('reached', 'cancelled'))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### A histogram time plot

Each time a target is reached, the time interval is registered in a vector which is then displayed as a histogram plot. This moment is also considered the start of a new goal reaching time interval, just like when pressing the **START** button.
> This plot is not animated! You have to rerun the cell manually if you want to update it!

In [22]:
fig3, ax3 = plt.subplots()
fig3.suptitle('Time to reach the targets', fontsize=16)
X=list(range(0,len(data),1))
fig3=plt.subplot(111)
plt.bar(X,data,width=0.5,color='b')
plt.ylabel('time (s)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Linear velocity line plot

In [14]:
def clb_cmdvel(msg):
    if len(cmdvel_linear) > 10:
        cmdvel_linear.pop(0)
    if len(cmdvel_angular) > 10:
        cmdvel_linear.pop(0)
    il = np.float32(msg.linear.x)
    ia = np.float32(msg.angular.z)
    cmdvel_linear.append(il)
    cmdvel_angular.append(ia)
    

jr.subscribe('/cmd_vel', Twist, clb_cmdvel)

In [15]:
fig4, ax4 = plt.subplots()
fig4.suptitle('Linear velocity', fontsize=16)

line1, = ax4.plot([], [], color ='r', label ='/odom')
line2, = ax4.plot([], [], color ='b', label ='/cmd_vel')
ax4.set_xlabel('time (s)') 
ax4.set_ylabel('velocity m/s')
ax4.legend(loc='upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
def init1():
    line1.set_data([], [])
    line2.set_data([], [])
    return [line1, line2]

def animate1(i):
    line1.set_data(time_arr, odom_linear)
    line2.set_data(time_arr, cmdvel_linear)
    ax4.axis([time_arr[0] - 0.2, time_arr[9] + 0.2, -1.0, 1.0])
    ax4.legend(loc='upper right')
    return [line1, line2]

anim1 = animation.FuncAnimation(fig4, animate1, init_func = init1,
                               frames = 100, interval=20, blit = False)

### Angular velocity line plot

In [17]:
fig5, ax5 = plt.subplots()
fig5.suptitle('Angular velocity', fontsize=16)

line3, = ax5.plot([], [], color ='r', label ='/odom')
line4, = ax5.plot([], [], color ='b', label ='/cmd_vel')
ax5.set_xlabel('time (s)') 
ax5.set_ylabel('velocity rad/s')
ax5.legend(loc='upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
def init2():
    line3.set_data([], [])
    line4.set_data([], [])
    return [line3, line4]

def animate2(i):
    line3.set_data(time_arr, odom_angular)
    line4.set_data(time_arr, cmdvel_angular)
    ax5.axis([time_arr[0] - 0.2, time_arr[9] + 0.2, -1.0, 1.0])
    ax5.legend(loc='upper right')
    return [line3, line4]

anim2 = animation.FuncAnimation(fig5, animate2, init_func = init2,
                               frames = 100, interval=20, blit = False)